In [105]:
import pandas as pd
import numpy as np
import json
import shapely
from shapely.ops import cascaded_union
import geog
import folium
from folium import FeatureGroup, LayerControl
from folium.plugins import *

In [106]:
df = pd.read_csv('shop_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Shop Category  290 non-null    object 
 1   Longitude      290 non-null    float64
 2   Latitude       290 non-null    float64
dtypes: float64(2), object(1)
memory usage: 6.9+ KB


In [107]:
# check the number of unique shop categories, and the unique labels
print('number of unique categories: ', df['Shop Category'].nunique())
print('unique categories: ', df['Shop Category'].unique())

number of unique categories:  2
unique categories:  ['Clothing' 'FoodandBeverage']


In [111]:
# we're going to create a map with a different shape per category
# define a function to create geometries for our map
def createShape(lat,lng,shape_points,distance_radius_meters):
    p = (lng, lat)
    n_points = shape_points+1
    d = distance_radius_meters  # meters
    angles = np.linspace(0, 360, n_points)
    polygon = geog.propagate(p, angles, d)
    return shapely.geometry.Polygon(polygon)

# we'll create hexagons and squares with 500meter sides
hexagon_side=6
square_side=4
shape_length_meters=500

# create a column to map shapes to shop categories
df['Shape Points'] = df['Shop Category'].map({'Clothing': hexagon_side, 'FoodandBeverage': square_side})

# create a column with the shape geometry
df['Shape Geometry'] = np.vectorize(createShape)(df['Latitude'], df['Longitude'], df['Shape Points'], shape_length_meters)

In [270]:
# create a base map with typical openstreetmap tiles
start_coords = [-26.043555, 28.019104]
base_map = folium.Map(location = start_coords, zoom_start = 13, tile='openstreetmap')

# overlay satellite images on basic map tiles
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Image',
        overlay = True,
        control = True,
        opacity=0.5
       ).add_to(base_map)

# create a dictionary with the colour hex codes for each category
color_dict={'Clothing':'#E81EC0','FoodandBeverage':'#9300FF'}

# add the first layer of polygons to the map, with overlapping polygons having borders removed
shopcategory = df['Shop Category'].unique()[0]
lgd_txt = '<span style="color: {col};">{txt}</span>'
color=color_dict[shopcategory]
feature_group = FeatureGroup(name=lgd_txt.format(txt=shopcategory, col=color))
polygons = df[df['Shop Category'] == shopcategory]['Shape Geometry'].to_list()
u = shapely.ops.unary_union(polygons)
highlight_function = lambda x: {'fillColor': color, 'color': 'black', 'weight': 2, 'dashArray': '5, 5', 'fillOpacity': 0.05}
style_function = lambda x: {'fillColor': color, 'color': color, 'weight': 3}
feature_group.add_child(folium.GeoJson(json.dumps(shapely.geometry.mapping(u)), style_function=style_function, highlight_function=highlight_function, overlay=False))
base_map.add_child(feature_group) 

# add the second layer of polygons to the map, with overlapping polygons having borders removed
shopcategory = df['Shop Category'].unique()[1]
lgd_txt = '<span style="color: {col};">{txt}</span>'
color2=color_dict[shopcategory]
feature_group = FeatureGroup(name=lgd_txt.format(txt=shopcategory, col=color2))
polygons = df[df['Shop Category'] == shopcategory]['Shape Geometry'].to_list()
u = shapely.ops.unary_union(polygons)
highlight_function = lambda x: {'fillColor': color2, 'color': 'black', 'weight': 2, 'dashArray': '5, 5', 'fillOpacity': 0.05}
style_function = lambda x: {'fillColor': color2, 'color': color2, 'weight': 3}
feature_group.add_child(folium.GeoJson(json.dumps(shapely.geometry.mapping(u)), style_function=style_function, highlight_function=highlight_function, overlay=False))
base_map.add_child(feature_group) 

# add some markers with a basic html pop-up when they're clicked, for the cities below
town_dict = {}
town_dict['Cape Town'] = (-33.926, 18.423)
town_dict['Durban'] = (-29.858 , 31.029)
town_dict['Johannesburg'] = (-26.202 , 28.044)
town_dict['Soweto'] = (-26.268 , 27.858)
town_dict['Pretoria'] = (-25.745 , 28.188)
town_dict['Port Elizabeth'] = (-33.961 , 25.615)
town_dict['East London'] = (-33.015 , 27.912)
town_dict['Bloemfontein'] = (-29.121 , 26.214)
feature_group = FeatureGroup(name="Major City")
for key in town_dict.keys():
    html = """<html><body style="font-family: sans-serif; font-size: 8pt"><center>Lat:{txt1:.3f}, Lon:{txt2:.3f}<br>City Name: {txt3}</body></html>""".format(txt1=town_dict[key][0], txt2=town_dict[key][1], txt3=key)
    iframe = folium.IFrame(html=html, width=150, height=60)
    feature_group.add_child(folium.Marker([town_dict[key][0], town_dict[key][1]], icon=folium.Icon(color="darkred",icon="asterisk", prefix='fa'), popup=folium.Popup(iframe)))
base_map.add_child(feature_group)

# add functionality for users to draw polygons and measure distances
fmtr = "function(num) {return L.Util.formatNum(num, 3);};"
MousePosition(position='topright', separator=' | ', prefix="Mouse:", lat_formatter=fmtr, lng_formatter=fmtr).add_to(base_map)
base_map.add_child(MeasureControl(active_color = '#EBF215', completed_color = '#EEF3F6'))

# add a layer legend and toggle to the map
folium.LayerControl().add_to(base_map)

# save the map as an html file
base_map.save("./shopsinsouthafrica.html")